In [396]:
import torch
import torch.nn as nn
import numpy as np

# Studying memory layout in pytorch (and numpy)

## Create simple linear layer and test vectors

In [59]:
layer = nn.Linear(3,3)

In [60]:
x0 = torch.randn(3)

In [61]:
y0 = layer(x0)

## Reproduce layer with Matrix multiplication

In [66]:
layer.weight.data

tensor([[-0.0305,  0.0748, -0.2738],
        [-0.1138, -0.4028,  0.1255],
        [ 0.5401,  0.4337,  0.1585]])

In [71]:
layer.weight.data@x0+layer.bias.data-y0

tensor([0., 0., 0.], grad_fn=<SubBackward0>)

## Matrix is given in row-major format

In [72]:
layer.weight.data.reshape(-1)

tensor([-0.0305,  0.0748, -0.2738, -0.1138, -0.4028,  0.1255,  0.5401,  0.4337,
         0.1585])

## The fast index is the rightmost

In [74]:
layer.weight.data[0,:]

tensor([-0.0305,  0.0748, -0.2738])

## Matrix multiplication is performed along fast index

In [78]:
sum(layer.weight.data[0,:] * x0) + layer.bias.data[0]

tensor(0.4191)

In [80]:
y0[0]

tensor(0.4191, grad_fn=<SelectBackward0>)

# Prepare ANN for MOM6

In [210]:
from torch.nn import functional as F
class ANN(nn.Module):
    def __init__(self, layer_sizes=[3, 17, 27, 5]):
        super().__init__()
        
        self.layer_sizes = layer_sizes
        

        layers = []
        for i in range(len(layer_sizes)-1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
        
        self.layers = layers
    
    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            if i < len(self.layers)-1:
                x = F.relu(x)
        return x

# Export neural network
The netcdf should contains the following information:
* Integer (int32) 'num_layers'
* Integer (int32) array 'layer_sizes'
* Float (float32 or float64) array input_norms
* Float array output_norms
* Every layer is matrix A0 (in column-major order) and bias b0, where 0 is the layer number
Optionally, for testing, there should be one sample prediction:
* Float array x_test with inputs
* Float y_test with outputs

In [388]:
import os
def export_ANN(ann, input_norms, output_norms, filename='ANN_test.nc'):
    ds = xr.Dataset()
    ds['num_layers'] = len(ann.layer_sizes)
    ds['layer_sizes'] = xr.DataArray(ann.layer_sizes, dims=['nlayers'])
    ds = ds.astype('int32') # MOM6 reads only int32 numbers
    
    for i in range(len(ann.layers)):
        # Naming convention for weights and dimensions
        matrix = f'A{i}'
        bias = f'b{i}'
        ncol = f'ncol{i}'
        nrow = f'nrow{i}'
        layer = ann.layers[i]
        
        # Transposed, because torch is row-major, while Fortran is column-major
        ds[matrix] = xr.DataArray(layer.weight.data.T, dims=[ncol, nrow])
        ds[bias] = xr.DataArray(layer.bias.data, dims=[nrow])
    
    # Save true answer for random vector for testing
    x0 = torch.randn(ann.layer_sizes[0])
    y0 = ann(x0 / input_norms) * output_norms
    nrow = f'nrow{len(ann.layers)-1}'
    
    ds['x_test'] = xr.DataArray(x0.data, dims=['ncol0'])
    ds['y_test'] = xr.DataArray(y0.data, dims=[nrow])
    
    ds['input_norms']  = xr.DataArray(input_norms.data, dims=['ncol0'])
    ds['output_norms'] = xr.DataArray(output_norms.data, dims=[nrow])

    
    print('x_test = ', ds['x_test'].data)
    print('y_test = ', ds['y_test'].data)
    
    if os.path.exists(filename):
        print(f'Rewrite {filename} ?')
        input()
        os.system(f'rm -f {filename}')
    
    ds.to_netcdf(filename)

# Generate ANN with random weights for testing

In [389]:
ann = ANN()
input_norms = torch.rand(3)
output_norms = torch.rand(5)

In [390]:
export_ANN(ann, input_norms, output_norms, '/scratch/pp2681/mom6/regression_tests/R4/ANN_test.nc')

x_test =  [0.28201845 0.1450909  1.1256554 ]
y_test =  [-0.01058591  0.26826918 -0.12749074  0.28866747  0.30501342]
Rewrite /scratch/pp2681/mom6/regression_tests/R4/ANN_test.nc ?



In [391]:
xr.open_dataset('/scratch/pp2681/mom6/regression_tests/R4/ANN_test.nc')

<xarray.Dataset>
Dimensions:       (nlayers: 4, ncol0: 3, nrow0: 17, ncol1: 17, nrow1: 27,
                   ncol2: 27, nrow2: 5)
Dimensions without coordinates: nlayers, ncol0, nrow0, ncol1, nrow1, ncol2,
                                nrow2
Data variables:
    num_layers    int32 ...
    layer_sizes   (nlayers) int32 ...
    A0            (ncol0, nrow0) float32 ...
    b0            (nrow0) float32 ...
    A1            (ncol1, nrow1) float32 ...
    b1            (nrow1) float32 ...
    A2            (ncol2, nrow2) float32 ...
    b2            (nrow2) float32 ...
    x_test        (ncol0) float32 ...
    y_test        (nrow2) float32 ...
    input_norms   (ncol0) float32 ...
    output_norms  (nrow2) float32 ...